In [1]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import expon, lognorm, loguniform, randint, uniform

# Logistic Regression

In [2]:
df = pd.read_csv("data/heart_disease.csv")
df.head(5)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [16]:
df_target = df["TenYearCHD"]

In [17]:
df = df.select_dtypes(include=["float"])
df = pd.concat([df, df_target], axis=1)
df = df.dropna(axis=0)
df.head(5)

,education,cigsPerDay,BPMeds,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,4.0,0.0,0.0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,2.0,0.0,0.0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1.0,20.0,0.0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,3.0,30.0,0.0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,3.0,23.0,0.0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [18]:
df.shape

(3656, 10)

In [19]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)
X_train, y_train = train_df.drop(columns=["TenYearCHD"]).values, train_df["TenYearCHD"].values
X_test, y_test = test_df.drop(columns=["TenYearCHD"]).values, test_df["TenYearCHD"].values
print(X_train.shape, X_test.shape)

(2924, 9) (732, 9)


In [6]:
import json

notebook_path = '/Users/jennylee/Library/Mobile Documents/com~apple~CloudDocs/Documents/Personal/ml-book/book/example.ipynb'

try:
    with open(notebook_path, 'r') as f:
        content = f.read()
        json.loads(content)  # This will raise an error if the content is not valid JSON
    print("Notebook JSON is valid.")
except json.JSONDecodeError as e:
    print(f"Notebook JSON is not valid: {e}")
except FileNotFoundError:
    print(f"Notebook file not found: {notebook_path}")

Notebook JSON is not valid: Expecting value: line 1 column 1 (char 0)


In [20]:
lr = LogisticRegression()
scores = cross_validate(lr, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

,fit_time,score_time,test_score,train_score
0,0.031070,0.000837,0.844444,0.845233
1,0.013518,0.000349,0.846154,0.847371
2,0.010750,0.000383,0.846154,0.849508
3,0.011033,0.000527,0.847863,0.846088
4,0.010779,0.000304,0.847603,0.848291


In [23]:
col_list = list(df.columns)[:-1]
col_list

['education',
 'cigsPerDay',
 'BPMeds',
 'totChol',
 'sysBP',
 'diaBP',
 'BMI',
 'heartRate',
 'glucose']

In [24]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print("Model weights: %s" % (lr.coef_))  # these are the learned weights
print("Model intercept: %s" % (lr.intercept_))  # this is the bias term
data = {"features": col_list, "coefficients": lr.coef_[0]}
pd.DataFrame(data)

Model weights: [[-0.14313691  0.02384328  1.41992278  0.00237223  0.0247263  -0.01374207
   0.0026058  -0.01492486  0.00614772]]
Model intercept: [-3.94592093]


/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,features,coefficients
0,education,-0.143137
1,cigsPerDay,0.023843
2,BPMeds,1.419923
3,totChol,0.002372
4,sysBP,0.024726
5,diaBP,-0.013742
6,BMI,0.002606
7,heartRate,-0.014925
8,glucose,0.006148


In [34]:
pd.DataFrame(X_train, columns=col_list)

,education,cigsPerDay,BPMeds,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,4.0,0.0,0.0,280.0,114.0,82.0,23.96,70.0,84.0
1,1.0,20.0,0.0,247.0,134.0,96.0,32.47,80.0,72.0
2,3.0,30.0,0.0,363.0,140.0,87.0,26.44,95.0,79.0
3,2.0,0.0,0.0,238.0,118.0,80.0,33.19,75.0,76.0
4,2.0,0.0,0.0,194.0,148.0,89.0,23.48,100.0,101.0
...,...,...,...,...,...,...,...,...,...
2919,1.0,0.0,0.0,175.0,173.0,59.0,27.99,70.0,75.0
2920,1.0,20.0,0.0,182.0,117.0,78.0,22.15,72.0,59.0
2921,3.0,0.0,0.0,159.0,108.0,72.0,27.68,70.0,84.0
2922,2.0,15.0,0.0,200.0,118.0,87.0,21.28,73.0,71.0


In [37]:
preprocessor = make_column_transformer(
    (StandardScaler(), col_list)
)

X_train = pd.DataFrame(X_train, columns=col_list)
y_train =  pd.DataFrame(y_train, columns=["TenYearCHD"])
pipe = make_pipeline(preprocessor, lr)
# pipe.fit(X_train, pd.DataFrame(y_train))

In [39]:
param_dist = {
    "logisticregression__C": uniform(0.1, 1e4),  # loguniform(1e-3, 1e3)
}

random_search = RandomizedSearchCV(pipe,                                    
                  param_distributions = param_dist, 
                  n_iter=100, 
                  n_jobs=-1, 
                  return_train_score=True)

# Carry out the search
random_search.fit(X_train, y_train)

/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jennylee/miniforge3/envs/575/lib/python3.12/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vecto

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['education',
                                                                                'cigsPerDay',
                                                                                'BPMeds',
                                                                                'totChol',
                                                                                'sysBP',
                                                                                'diaBP',
                                                                                'BMI',
                                                                                'heartRate',
                                                                                'glucose'])])),
                                             ('logisticregression',
                                              LogisticRegression())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'logisticregression__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x153bb05f0>},
                   return_train_score=True)

In [40]:
random_search.best_score_

0.8502048940405105

In [41]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_score",
        "param_logisticregression__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

rank_test_score,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
mean_test_score,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,...,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205,0.850205
param_logisticregression__C,1409.436222,3091.017825,9505.378046,3996.095650,4220.744994,2445.333951,1715.020199,5198.610576,1761.380918,9858.592578,...,6511.565612,3264.177112,4872.097331,9979.077633,4726.829504,8355.337900,2305.441786,7274.168100,242.159866,817.800357
mean_fit_time,0.006232,0.006463,0.007208,0.006558,0.006378,0.006848,0.006662,0.006170,0.006826,0.007298,...,0.004949,0.005755,0.004971,0.004929,0.006235,0.005830,0.005249,0.006109,0.004474,0.005854
